In [40]:
import numpy as np
import itertools
import cProfile
from collections import defaultdict
from copy import deepcopy

import json
EXAMPLE="example.txt"
EXAMPLE2="example2.txt"
INPUT="input.txt"
rotation=[[(0,1),(1,1),(2,1)],[(0,1),(1,-1),(1,-1)],
          [(0,1),(2,1),(1,-1)],[(0,1),(2,-1),(1,1)],
          [(1,1),(0,1),(2,-1)],[(1,1),(0,-1),(2,1)],
          [(1,1),(2,1),(0,1)],[(1,1),(2,-1),(0,-1)],
          [(2,1),(0,1),(1,1)],[(2,1),(0,-1),(1,-1)],
          [(2,1),(1,1),(0,-1)],[(2,1),(1,-1),(0,1)],
          [(0,-1),(1,1),(2,-1)],[(0,-1),(1,-1),(2,1)],
          [(0,-1),(2,1),(1,1)],[(0,-1),(2,-1),(1,-1)],
          [(1,-1),(0,1),(2,1)],[(1,-1),(0,-1),(2,-1)],
          [(1,-1),(2,1),(0,-1)],[(1,-1),(2,-1),(0,1)],
          [(2,-1),(0,1),(1,-1)],[(2,-1),(0,-1),(1,1)],
          [(2,-1),(1,1),(0,1)],[(2,-1),(1,-1),(0,-1)]]




#rotation=[[(1,1),(0,1),(2,-1)],[(1,1),(2,1),(0,1)],[(1,1),(0,-1),(2,1)],[(1,1),(2,-1),(2,-1)],
#          [(2,-1),(1,1),(0,1)],[(0,1),(1,1),(2,1)],[(2,1),(1,1),(0,-1)],[(0,-1),(1,1),(2,-1)],
#          [(2,1),(0,1),(1,1)],[(0,-1),(2,1),(1,1)],[(2,-1),(0,-1),(1,1)],[(0,1),(2,-1),(1,1)],
#          [(0,-1),(2,1),(1,-1)],[(2,-1),(0,-1),(1,-1)],[(0,1),(2,-1),(1,-1)],[(2,1),(0,1),(1,-1)],
#          [(1,-1),(0,1),(2,1)],[(1,-1),(2,-1),(0,1)],[(1,-1),(0,-1),(2,-1)],[(1,-1),(2,1),(0,-1)],
#          [(2,1),(1,-1),(0,1)],[(0,1),(1,-1),(2,-1)],[(2,-1),(1,-1),(0,-1)],[(0,-1),(1,-1),(2,1)]]

#rotation=[[y,x,-z],[y,z,x],[y,-x,z],[y,-z,-z],
#          [-z,y,x],[x,y,z],[z,y,-x],[-x,y,-z],
#          [z,x,y],[-x,z,y],[-z,-x,y],[x,-z,y],
#          [-x,z,-y],[-z,-x,-y],[x,-z,-y],[z,x,-y],
#          [-y,x,z],[-y,-z,x],[-y,-x,-z],[-y,z,-x],
#          [z,-y,x],[x,-y,-z],[-z,-y,-x],[-x,-y,z]]

def doARotation(elem,n):
    currentRotation=rotation[n]
    x = elem[currentRotation[0][0]]*currentRotation[0][1]
    y = elem[currentRotation[1][0]]*currentRotation[1][1]
    z = elem[currentRotation[2][0] ]*currentRotation[2][1]
    return [x,y,z]
def getAllRotation(listElem):
    allRot=[]
    for n,e in enumerate(rotation):
        rot=[]
        for elem in listElem:
            rot.append(doARotation(elem,n))
        allRot.append(rot)
    return np.array(allRot)
#elem=[[-1,-1,1],[-2,-2,2],[-3,-3,3],[-2,-3,1],[5,6,-4],[8,0,7]]
#getAllRotation(elem)


def revertRotation(elem,num):
    nRotation=rotation[num].copy()
    #print(nRotation)
    reverseDict={}
    for i,e in enumerate(elem):
        #print(str(e),"*",str(nRotation[i][1]),"=",e*nRotation[i][1])
        reverseDict[nRotation[i][0]]=e*nRotation[i][1]
    return np.array([reverseDict[0],reverseDict[1],reverseDict[2]])
    

with open(INPUT) as f:
    input_=f.read().strip().split("\n\n")
    input_=[x.splitlines() for x in input_]
    input_=np.array([[np.array(list(map(int,y.split(",")))) for y in x[1:]] for x in input_],dtype=object)
#print(np.array(input_))
scannerDict={}
for i,element in enumerate(input_):
    scannerDict[i]=np.array(element)
#print(scannerDict[1])


def checkIntersection(couple,scannerDict):
    #print("\n\n\n\n\ncouple ",couple)
    scanner1=scannerDict[couple[0]]
    scanner2=scannerDict[couple[1]]
    scannerRot=getAllRotation(scanner2)
    #print("\n"*2,"ROTAL\n",scannerRot,"end\n"*2)
    for beacon1 in scanner1:
        for num,rot in enumerate(scannerRot):
            for rotatedBeacon2 in rot:
                difference= beacon1-rotatedBeacon2
                comparison= [x+difference for x in rot]
                #print(difference,rotatedBeacon,beacon2,"\n"*2,scanner1,"\n"*2,comparison,"\n"*2,scanner2,"\n"*2)
                size=0
                intersect= set(map(tuple, comparison)) & set(map(tuple, scanner1))
                intersect= list(map(list, intersect))
                #print("scanner2 comparison","\n"*2,np.array(scanner2),"\n"*2,np.array(comparison))
                #print("intersect",intersect,"\n")
                #print(intersect.size)
                if(len(intersect)>=12):
                    #print("found",couple,len(intersect),"\n"*2,np.array(intersect))
                    print("found",couple,difference,num)
                    return True,difference,num
                
        #print("\n"*4)
    return False,None,None

def jg(scannerDict):
    foundScanners=[0]
    remain=[x for x in scannerDict if not x==0]
    intersectionDict=defaultdict(lambda: defaultdict(defaultdict))
    combinaison=[]
    keys = list(scannerDict.keys())
    intersection=[]
    for pair in itertools.permutations(keys, r=2):
        combinaison.append(pair)
    #print(combinaison)
    while(remain):
        newFoundScanner=[]
        #print(len(remain))
        for i,x in enumerate(foundScanners):
            for j,y in enumerate(remain):   
                inter,difference,num=checkIntersection((x,y),scannerDict)
                if(inter):
                    intersection.append((x,y))
                    intersectionDict[y][x]["difference"]=list(difference.tolist())
                    intersectionDict[y][x]["num"]=int(num)
                    newFoundScanner.append((y,j,num))
        if(newFoundScanner):
            for i,elem in enumerate(newFoundScanner):
                print(elem)
                foundScanners.append(elem[0])
                remain.pop(elem[1]-i)
                scannerDict[elem[0]]= np.array([doARotation(x,elem[2]) for x in scannerDict[elem[0]]])

    #print(json.dumps(intersectionDict,indent=4))
    return intersection,intersectionDict


def getPath(num,intersectionDict,visited):
    path=[]
    visited.append(num)
    if(0 in intersectionDict[num]):
        path.append(0)
        path.append(num)
        return path
    for elem in intersectionDict[num]:
        if(elem in visited):
            continue
        visitedTemp=visited.copy()
        visitedTemp.append(elem)
        partialPath=getPath(elem,intersectionDict,visitedTemp)
        if(partialPath):
            if(isinstance(partialPath,list)):
                path=path+partialPath
            else:
                path.append(partialPath)
            visited.append(elem)
    if(path):
        path.append(num)

    return path

intersection,intersectionDict=jg(scannerDict)
#getPath(4,intersectionDict)


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), 

found (0, 27) [1382  -30  -20] 4
found (0, 36) [  172     2 -1300] 8
(27, 26, 4)
(36, 35, 8)
37
found (27, 19) [ -102 -1196   -20] 17
found (27, 21) [1095  -17   22] 16
found (27, 28) [-115   72 1132] 7
found (27, 35) [-109 1138   98] 15
found (36, 32) [  -22    58 -1114] 0
(19, 18, 17)
(21, 20, 16)
(28, 26, 7)
(35, 33, 15)
(32, 30, 0)
32
found (36, 32) [  -22    58 -1114] 0
found (19, 34) [   87 -1227   -17] 13
found (21, 13) [   13    42 -1184] 15
found (21, 33) [ -27  139 1128] 8
found (28, 33) [1183   50   18] 8
found (28, 39) [ -14  -28 1309] 11
found (35, 18) [  48 1227  -29] 18
found (32, 32) [0 0 0] 0
(32, 26, 0)
(34, 28, 13)
(13, 12, 15)
(33, 27, 8)
(33, 27, 8)
(39, 31, 11)
(18, 17, 18)
(32, 26, 0)
24
found (21, 33) [ -27  139 1128] 6
found (28, 33) [1183   50   18] 6
found (35, 18) [  48 1227  -29] 0
found (34, 8) [1101  -29  112] 11
found (34, 38) [    3    53 -1108] 4
found (13, 5) [  -10     2 -1276] 6
found (13, 15) [1134   66  -96] 22
found (13, 17) [  75 1127  -36] 14
f

IndexError: pop index out of range

In [39]:
vis=[]
getPath(3,intersectionDict,vis)
import functools
import operator
#[  -20+1105, -1133+(-72)  1061+168]
def calculatePosition(intersectionDict,scannerDict,num):
    vis=[]
    #print(json.dumps(intersectionDict,indent=4))
    path=getPath(num,deepcopy(intersectionDict),vis)
    path= [path] if isinstance(path,int) else path
    #print("path",path)
    summ=np.array([0,0,0])
    for i in range(0,len(path)-1):
        #print(i,i+1)
        summ+=intersectionDict[path[i+1] ][path[i]]["difference"]
    return summ

updatedScanners={}
updatedScanners[0]=deepcopy(scannerDict[0])
for k in scannerDict:
    if(k==0):
        continue
    summ=calculatePosition(deepcopy(intersectionDict),deepcopy(scannerDict),k)
    kList=deepcopy(scannerDict[k])
    kList=np.array([x+summ for x in kList])
    updatedScanners[k]=kList
    
    
calculatePosition(deepcopy(intersectionDict),deepcopy(scannerDict),3)
#for k in scannerDict:
finalList=[]
for k in updatedScanners:
    finalList+=deepcopy(updatedScanners[k]).tolist()
finalList=list(map(list, set(map(tuple, map(set, finalList)))))
print(len(finalList))
#print(finalList)
#cou=0
#for elem in finalList:
#    cou+=1
#print(cou)



#a=np.array(intersectionDict[1][0]["difference"])

#b=np.array(intersectionDict[4][1]["difference"])
#b=revertRotation(b,12)
#c=np.array(intersectionDict[4][2]["difference"])
#c=b=revertRotation(c,4)
#print(a,b,c)
#print(a+b+c)
# -y    -z x
#[1125,-72,168]
#before [168,-1125,72]
#-x , y , -z [-168,1125,-72]
#y -z -x [1125,72,168]

79


In [302]:
#r1=revertRotation([-68,1246,43],12)
r1=np.array([68,-1246,-43])
r2=revertRotation([ 88,113,-1104] ,12)
print(r1,r2,"\n",r1+r2)
#np.array([-68,1246,43])-np.array([ -88,-113,1104])
np.array([1105,-1205,1229])-np.array([-20,-1133,1061])
#[1125,  -72,  168]
revertRotation(revertRotation(revertRotation( [168,-1125,72],12 ),7),4)

[   68 -1246   -43] [ -88  113 1104] 
 [  -20 -1133  1061]


array([ -168,    72, -1125])

In [16]:
np.array([68,-1246,-43])+ np.array([-88,113,1104])+ np.array([1125,-72,168])

array([ 1105, -1205,  1229])

In [53]:
#np.array([0,2])-np.array([-1,-1])
maximum=len(input_)
my_list = list(range(1,maximum+1))
for pair in itertools.combinations(my_list, r=2):
    print(pair)
list(scannerDict.keys())

(1, 2)
(1, 3)
(1, 4)
(1, 5)
(2, 3)
(2, 4)
(2, 5)
(3, 4)
(3, 5)
(4, 5)


[1, 2, 3, 4, 5]

In [11]:
a=[[1,2,3],[2,8,9]]
b=[[1,2,3],[8,7,3]]
intersection_set = set(map(tuple, a)) & set(map(tuple, b))
list(map(list, intersection_set))

[[1, 2, 3]]